In [25]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from binance.client import Client
import pandas as pd

# For public price data, you can leave keys empty
client = Client(api_key='', api_secret='')

# Example: Get 100 daily candles for BTC/USDT
klines = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1DAY, limit=100)

# Convert to DataFrame
df = pd.DataFrame(klines, columns=[
    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close time', 'Quote asset volume', 'Number of trades',
    'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
])
df['Date'] = pd.to_datetime(df['Open time'], unit='ms')
df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]


In [27]:
print(df.head())

        Date      Open      High       Low     Close       Volume
0 2025-02-14  96608.13  98826.00  96252.82  97500.48  18173.02646
1 2025-02-15  97500.47  97972.26  97223.58  97569.66   7349.37683
2 2025-02-16  97569.67  97704.47  96046.18  96118.12   8191.42490
3 2025-02-17  96118.12  97046.59  95205.00  95780.00  16492.04510
4 2025-02-18  95780.01  96753.91  93388.09  95671.74  23368.19471


In [28]:
pip install pandas-ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
pip install ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import ta

df = df.dropna()
df['RSI_14'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()
df['SMA_10'] = ta.trend.SMAIndicator(close=df['Close'], window=10).sma_indicator()
df['EMA_10'] = ta.trend.EMAIndicator(close=df['Close'], window=10).ema_indicator()


# Bollinger Bands
bb = ta.volatility.BollingerBands(close=df['Close'])
df['BB_High'] = bb.bollinger_hband()
df['BB_Low'] = bb.bollinger_lband()

# MACD
macd = ta.trend.MACD(close=df['Close'])
df['MACD'] = macd.macd()

# Stochastic Oscillator
stoch = ta.momentum.StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'])
df['Stoch_K'] = stoch.stoch()
df['Stoch_D'] = stoch.stoch_signal()

# CCI (Commodity Channel Index)
cci = ta.trend.CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'])
df['CCI'] = cci.cci()

# ADX (Average Directional Index)
adx = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'])
df['ADX'] = adx.adx()



In [31]:
print(df.tail(10))  # shows last 10 rows

         Date       Open       High        Low      Close        Volume  \
90 2025-05-15  103507.83  104192.70  101383.07  103763.71  17998.986040   
91 2025-05-16  103763.71  104550.33  103100.49  103463.90  15683.880240   
92 2025-05-17  103463.90  103709.86  102612.50  103126.65  11250.896220   
93 2025-05-18  103126.65  106660.00  103105.09  106454.26  21599.987260   
94 2025-05-19  106454.27  107108.62  102000.00  105573.74  30260.035240   
95 2025-05-20  105573.73  107320.00  104184.72  106849.99  23705.482750   
96 2025-05-21  106850.00  110797.38  106100.01  109643.99  45531.040345   
97 2025-05-22  109643.99  111980.00  109177.37  111696.21  31630.773130   
98 2025-05-23  111696.22  111800.00  106800.00  107318.30  31737.723090   
99 2025-05-24  107318.30  107680.83  106875.41  107363.32    859.642510   

       RSI_14      SMA_10         EMA_10        BB_High        BB_Low  \
90  69.673543  102319.221  101993.989296  107067.052731  90258.375269   
91  68.451737  102982.209  1

In [32]:
pip install newsapi-python vaderSentiment

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd


newsapi = NewsApiClient(api_key='fe27f943772a4235a29f751edabae735')

all_articles = newsapi.get_everything(
    q='Bitcoin',
    language='en',
    sort_by='publishedAt',
    page_size=100
)
news_df = pd.DataFrame(all_articles['articles'])
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

# Sentiment analysis using VADER
analyzer = SentimentIntensityAnalyzer()
news_df['sentiment_score'] = news_df['title'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Add 'Date' column for grouping
news_df['Date'] = news_df['publishedAt'].dt.date
news_df['Date'] = pd.to_datetime(news_df['Date'])

# Group by date to get average sentiment per day
daily_sentiment = news_df.groupby('Date')['sentiment_score'].mean().reset_index()
daily_sentiment.rename(columns={'sentiment_score': 'Sentiment'}, inplace=True)


In [34]:
news_df.head()


,source,author,title,description,url,urlToImage,publishedAt,content,sentiment_score,Date
0,"{'id': None, 'name': 'Biztoc.com'}",bloomberg.com,Trump Team’s ‘Game of Thrones’ on Venezuela Wh...,,https://biztoc.com/x/35f84574501cde74,https://biztoc.com/cdn/35f84574501cde74_s.webp,2025-05-23 00:40:39+00:00,"{ window.open(this.href, '_blank'); }, 200); r...",0.0000,2025-05-23
1,"{'id': None, 'name': 'Biztoc.com'}",bloomberg.com,Europe-Japan Fighter Jet Races Against China’s...,,https://biztoc.com/x/20a44f5ac3e71383,https://biztoc.com/cdn/20a44f5ac3e71383_s.webp,2025-05-23 00:40:18+00:00,"{ window.open(this.href, '_blank'); }, 200); r...",0.5267,2025-05-23
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Shomik Sen Bhattacharjee,Kevin O'Leary Says This One Habit Alone Could ...,Kevin O'Leary says the simplest path to millio...,https://finance.yahoo.com/news/kevin-oleary-sa...,https://media.zenfs.com/en/Benzinga/f5a639937b...,2025-05-23 00:31:34+00:00,Kevin O'Leary says the simplest path to millio...,-0.5267,2025-05-23
3,"{'id': None, 'name': 'newsBTC'}",Ash Tiwari,Bitcoin ATH Met With Mixed Whale Reactions – W...,"Yesterday, Bitcoin (BTC) made a fresh all-time...",http://www.newsbtc.com/bitcoin-news/bitcoin-at...,https://www.newsbtc.com/wp-content/uploads/202...,2025-05-23 00:30:39+00:00,Strict editorial policy that focuses on accura...,0.0000,2025-05-23
4,"{'id': None, 'name': 'Biztoc.com'}",bloomberg.com,Japan’s Akazawa to Visit US for Third Round of...,,https://biztoc.com/x/72a88e6e856eb1ae,https://biztoc.com/cdn/72a88e6e856eb1ae_s.webp,2025-05-23 00:29:26+00:00,"{ window.open(this.href, '_blank'); }, 200); r...",0.0000,2025-05-23


In [ ]:
# Convert dates to same format
daily_sentiment['Date'] = pd.to_datetime(daily_sentiment['Date'])

# Merge on 'Date'
df = pd.merge(df, daily_sentiment, on='Date', how='left')

#fill for na
df['Sentiment'] = df['Sentiment'].fillna(0)


In [36]:
print(df[['Date', 'Close', 'Sentiment']].tail(10))  # last 10 rows
print(df.describe())  # numerical summary (mean, std, min, etc.)

         Date      Close  Sentiment
90 2025-05-15  103763.71   0.000000
91 2025-05-16  103463.90   0.000000
92 2025-05-17  103126.65   0.000000
93 2025-05-18  106454.26   0.000000
94 2025-05-19  105573.74   0.000000
95 2025-05-20  106849.99   0.000000
96 2025-05-21  109643.99   0.000000
97 2025-05-22  111696.21   0.050504
98 2025-05-23  107318.30  -0.024421
99 2025-05-24  107363.32   0.000000
                      Date           Open          High            Low  \
count                  100     100.000000     100.00000     100.000000   
mean   2025-04-04 12:00:00   90844.469600   92510.96490   89052.400700   
min    2025-02-14 00:00:00   76322.420000   80867.99000   74508.000000   
25%    2025-03-10 18:00:00   84025.290000   85317.99750   82450.185000   
50%    2025-04-04 12:00:00   87445.520000   90207.07500   85659.545000   
75%    2025-04-29 06:00:00   96505.177500   97140.94750   95359.207500   
max    2025-05-24 00:00:00  111696.220000  111980.00000  109177.370000   
std         

In [37]:
#cretae a target variable
# Create binary target: 1 if tomorrow's close is higher than today's, else 0
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)


In [38]:
features = [
    'RSI_14', 'SMA_10', 'EMA_10', 'Sentiment',
    'BB_High', 'BB_Low', 'MACD', 'Stoch_K', 'Stoch_D', 'CCI', 'ADX'
]

df = df.dropna()  # drop rows with any NaN values

X = df[features]
y = df['Target']

In [39]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#Train/test Split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred)) 
print(classification_report(y_test, y_pred))


Accuracy: 0.5333333333333333
              precision    recall  f1-score   support

           0       0.67      0.25      0.36         8
           1       0.50      0.86      0.63         7

    accuracy                           0.53        15
   macro avg       0.58      0.55      0.50        15
weighted avg       0.59      0.53      0.49        15

